# GEE Landsat Visualization with Folium

Visualize Landsat imagery on an interactive folium map.

In [1]:
import ee
import folium
import requests

# Initialize GEE
ee.Initialize(project='tl-cities')
print('✅ GEE initialized successfully!')

✅ GEE initialized successfully!


## Helper Functions for Adding GEE Layers to Folium

In [2]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the Earth Engine layer method to folium
folium.Map.add_ee_layer = add_ee_layer

## Get Landsat Data and Create Visualization

In [3]:
# Define area of interest (London)
london = ee.Geometry.Point([-0.1278, 51.5074])
london_bounds = london.buffer(50000)  # 50km buffer around London

# Get recent Landsat 8 imagery
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
            .filterBounds(london_bounds) \
            .filterDate('2023-06-01', '2023-09-01') \
            .sort('CLOUD_COVER') \
            .first()

print(f'Found Landsat image from: {landsat.date().getInfo()}')
print(f'Cloud cover: {landsat.get("CLOUD_COVER").getInfo()}%')

Found Landsat image from: {'type': 'Date', 'value': 1685875557650}
Cloud cover: 0.14%


In [4]:
# Create the map centered on London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Visualization parameters for true color
vis_params_rgb = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

# Add true color Landsat layer
m.add_ee_layer(landsat, vis_params_rgb, 'Landsat 8 True Color')

# Visualization parameters for false color (infrared)
vis_params_false = {
    'bands': ['B5', 'B4', 'B3'],  # NIR, Red, Green
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

# Add false color layer
m.add_ee_layer(landsat, vis_params_false, 'Landsat 8 False Color (NIR)')

# Add layer control
folium.LayerControl().add_to(m)

m

## Land Surface Temperature Example

In [5]:
# Get MODIS Land Surface Temperature data
modis_lst = ee.ImageCollection('MODIS/061/MOD11A1') \
             .filterBounds(london_bounds) \
             .filterDate('2023-07-01', '2023-07-31') \
             .select('LST_Day_1km') \
             .mean()

# Convert from Kelvin to Celsius
lst_celsius = modis_lst.multiply(0.02).subtract(273.15)

# Create temperature map
m_temp = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Temperature visualization parameters
temp_vis = {
    'min': 20,
    'max': 40,
    'palette': ['blue', 'cyan', 'lime', 'yellow', 'red']
}

# Add temperature layer
m_temp.add_ee_layer(lst_celsius, temp_vis, 'Land Surface Temperature (°C)')

# Add colorbar/legend
colormap = folium.LinearColormap(
    colors=['blue', 'cyan', 'lime', 'yellow', 'red'],
    vmin=20, vmax=40,
    caption='Land Surface Temperature (°C)'
)
colormap.add_to(m_temp)

m_temp

In [ ]:
# Save the maps
m.save('../outputs/maps/london_landsat.html')
m_temp.save('../outputs/maps/london_temperature.html')

print('✅ Maps saved to outputs/maps/')
print('- london_landsat.html')
print('- london_temperature.html')